In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup  # import BeautifulSoup libraries



#import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
#from IPython.display import Image 
#from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
#from pandas.io.json import json_normalize
#import json # library to handle JSON files

# !conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans


#### Scrape and Dataframe

In [65]:
# Scrape of Canada postal codes
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

# dataframe creation with PostalCode, Borough, and Neighborhood
df = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])

    
# Filling the dataframe with all Postalcodes, Borough, and Neighborhood 
for i_row in table.find_all('tr'):
    row_data=[]
    for i_col in i_row.find_all('td'):
        row_data.append(i_col.text.strip())

    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [66]:
# Ignoring cells with a borough that is Not assigned
df=df[df['Borough']!='Not assigned']
df["Neighborhood"] = df["Neighborhood"].str.replace(" /",", ")
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [74]:
# Combining all neighborhood into one row with the same postalcode, separated with a comma
df2=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2=df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood2'},inplace=True)

df3 = pd.merge(df, df2, on='Postalcode')
df3.drop(['Neighborhood'],axis=1,inplace=True)
df3.drop_duplicates(inplace=True)
df3.rename(columns={'Neighborhood2':'Neighborhood'},inplace=True)

df3.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [75]:
df3.shape

(103, 3)